# Astronomical Data in Python

## Introduction

This is the series of Jupyter notebooks written by Allen Downey; see https://allendowney.github.io/AstronomicalData/README.html.

### Original Notebooks

I downloaded the original Jupyter notebooks to the following files:
- 01_query,ipynb
- 02_coords.ipynb
- 03_motion.ipynb
- 04_select.ipynb
- 05_join.ipynb
- 06_photo.ipynb
- 07_plot.ipynb

### Resources

#### Adrian Price-Whelan
https://adrian.pw/
Writes open source software for astronomy.
https://adrian.pw/blog/flexible-density-model-jax/
 
#### ADQL
The ADQL cookbook is located at https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook
 
#### Data Carpentries
This book is adapted from the Data Carpentries lessons located at https://datacarpentry.org/astronomy-python/
 
#### GAIA
- Obtain a Gaia user account: https://www.cosmos.esa.int/web/gaia-users/register
- https://www.ivoa.net/documents/VOTable/
- https://github.com/gaia-unlimited
- https://www.simonsfoundation.org/flatiron/

#### astropy
- https://learn.astropy.org/tutorials/gaia-galactic-orbits.html
- https://iopscience.iop.org/article/10.3847/1538-4357/ac7c74
- https://docs.astropy.org/en/latest/generated/examples/coordinates/plot_galactocentric-frame.html#sphx-glr-generated-examples-coordinates-plot-galactocentric-frame-py

#### astroquery
- https://astroquery.readthedocs.io/en/latest/index.html
- Gaia: https://astroquery.readthedocs.io/en/latest/gaia/gaia.html

#### galstreams

> Nearly a hundred stellar streams have been found to date around the Milky Way and the number keeps growing at an ever faster pace. Here we present the \galstreams\ library, a compendium of angular position, distance, proper motion and radial velocity tracks for nearly a hundred (\Nunique) Galactic stellar streams. The information published in the literature has been collated and homogenised in a consistent format and used to provide a set of features uniformly computed throughout the library: e.g. stream length, end points, mean pole, stream's coordinate frame, polygon footprint, and pole and angular momentum tracks. We also use the information compiled to analyse the distribution of several observables across the library and to assess where the main deficiencies are found in the characterisation of individual stellar streams, as a resource for future follow-up efforts. The library is intended to facilitate keeping track of new discoveries and to encourage the use of automated methods to characterise and study the ensemble of known stellar streams by serving as a starting point. The library is publicly available as a Python package and served at the galstreams GitHub repository.

- https://ui.adsabs.harvard.edu/abs/2017ascl.soft11010M/abstract
- doi:10.1093/mnras/stx2937
- https://arxiv.org/abs/2204.10326
- https://pypi.org/project/galstreams/
- https://github.com/cmateu/galstreams
- https://www.ascl.net/phpBB3/viewtopic.php?t=34496

#### ICRS
- http://localhost:8888/lab?token=b116a71836d50884c67b0ca55fbb800442d3bb793b5c0823

### Stellar Streams in the Milky Way
- https://www.syfy.com/syfy-wire/gaia-the-sky-is-littered-with-undigested-galaxies

## Set Up Virtual Environment

See https://jupyter.org/install for installing Jupyter Lab.

I installed astroquery 0.4.6.

```bash
cd
brew install pyenv
brew install rust
pyenv install 3.11.1
pyenv global 3.11.1
python3 -m venv .astro
source .astro/bin/activate
pip3 list
python3 -m pip install --upgrade pip
python3 -m pip install --upgrade setuptools
python3 -m pip install jupyterlab
python3 -m pip install astroquery

cd ~/OneDrive/Documents/astro
python3 -Xfrozen_modules=off -m jupyterlab
```

## Imports

In [ ]:
from astroquery.gaia import Gaia

## Queries

Despite what the book's text stated, importing astroquery.gaia did not write any output. However, the `Gaia.load_tables()` command worked correctly.

### List Tables

As shown, printing a table and not just its name prints the table's metadata.

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in tables:
    print(table.name)
print()
print(tables[0])

### Get Table Metadata

We will work with the following tables:
- gaiadr2.gaia_source
- gaiadr2.panstarrs1_original_valid
- gaiadr2.panstarrs1_best_neighbour

The tables for DR2 are documented here: https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html.

In [ ]:
data_table_names = [
    "gaiadr2.gaia_source",
    "gaiadr2.panstarrs1_original_valid",
    "gaiadr2.panstarrs1_best_neighbour"]
for data_table_name in data_table_names:
    meta = Gaia.load_table(data_table_name)
    print(meta)
    print()
    for column in meta.columns:
        print(column.name)
    print("\n##########\n")

### Execute a Synchronous Query

In [ ]:
# Start a query and fetch the results.
# The .launch_job() method starts a synchronous command that returns a
# maximum of 2000 rows.
query1 = """
    SELECT 
    TOP 10
        source_id,
        ra,
        dec,
        parallax 
    FROM
        gaiadr2.gaia_source
"""
job = Gaia.launch_job(query1)
print(type(job))
print(job)
print(dir(job))
print(job.jobid)
print()
results = job.get_results()
print(type(results))

In [ ]:
# Show the results and have Jupyter format them.
# Each column nas a name, unit type, and data type.
results

### Execute an Asynchronous Query

> You might notice that some values of parallax are negative. As this FAQ explains, “Negative parallaxes are caused by errors in the observations.” They have “no physical meaning,” but they can be a “useful diagnostic on the quality of the astrometric solution.”

In [ ]:
query2 = """
    SELECT 
    TOP 3000
        source_id,
        ra,
        dec,
        pmra,
        pmdec,
        parallax
    FROM
        gaiadr2.gaia_source
    WHERE
        parallax < 1
"""
job = Gaia.launch_job_async(query2, verbose=True)
print(job)

In [ ]:
results = job.get_results()
results

In [ ]:
# Remove the async job now that it's complete.
job_removed = Gaia.remove_jobs([job.jobid])
print("job_removed:", job_removed)

I stopped at the exercises about submitting bad queries and about modifying queries.